This is the analysis pipeline which will have to do the following:

-Read in the experimental data, and create the arrays for all stimuli and selected stimuli

-Create iterative composites which incrementally consider more participant judgements

-have these composites be judged by the ensemble model in a one-shot fashion

-compute the scores of the VVIQ & CAPS

-compute correlations between the VVIQ & CAPS

-compute correlations between the questionnaires and the experimental results

-compute the test retest reliability

-anything else you might remember from the notebook
